In [1]:
# Load packages
import IPython.display as ipd
import os
import json
import random
import pydub
import numpy as np
import torch
import copy
from torch.utils.data import DataLoader
from models import Wav2Letter
torch.cuda.set_device(1)
# Import 
from utils import AudioDataset, PostProcess
root_dir = '../../ssd/audio'

In [2]:
# Create a list of paths
kore_word_dir = os.path.join(root_dir,'kore_words')

from glob import glob
audio_dir = os.path.join(kore_word_dir,'word_audio_npy')
glob_pattern = os.path.join(audio_dir, '*')
audio_list = sorted(glob(glob_pattern), key=os.path.getctime)

target_dir = os.path.join(kore_word_dir,'targets')
glob_pattern = os.path.join(target_dir, '*')
target_list = sorted(glob(glob_pattern), key=os.path.getctime)

In [3]:
n_audio_max = 80000
n_target_max = 9
dataset = AudioDataset(audio_list,target_list,n_audio_max,n_target_max)
lookup_dict = json.load(open('./lookup.json'))
postprocessor = PostProcess(lookup_dict)
random_integer = random.randint(0,len(audio_list))
original_word_audio_dir =  os.path.join(kore_word_dir,'kore-sound-vocab-munged')

random_audio, random_target, _ = dataset[random_integer]
random_audio_ID = os.path.split(audio_list[random_integer])[-1][:-4]
original_audio_path = os.path.join(original_word_audio_dir,random_audio_ID) + '.mp3'

rs = int(pydub.utils.mediainfo(original_audio_path)['sample_rate'])

print(postprocessor.target2kana(random_target.cpu().numpy()))
ipd.display(ipd.Audio(random_audio.cpu().numpy()*2**15, rate=rs,autoplay=True))

シンド


In [4]:
n_dataset = len(audio_list)
train_proportion = .9
n_train = int(.9*n_dataset)
inds = np.arange(n_dataset)
np.random.shuffle(inds)
inds_train = inds[:n_train]
inds_val = inds[n_train:]
n_val = n_dataset - n_train

audio_list_train = np.array(audio_list)[inds_train].tolist()
audio_list_val = np.array(audio_list)[inds_val].tolist()

target_list_train = np.array(target_list)[inds_train].tolist()
target_list_val = np.array(target_list)[inds_val].tolist()

In [5]:
# I know that it possible to use the Subset class to separate the train and validation loaders,
# however I don't know how to make the validation dataset not randomly subsampled in that case

dataset_train = AudioDataset(audio_list_train,target_list_train,n_audio_max,n_target_max)
dataset_val = AudioDataset(audio_list_val,target_list_val,n_audio_max,n_target_max)

batch_size_train = 32
batch_size_val = 64

train_loader = DataLoader(dataset_train, batch_size=batch_size_train,shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size_val)

In [6]:
def train(model,train_loader,val_loader,optimizer,criterion,n_epoch) :
    
    optimal_model = copy.copy(model)
    av_lev_dist_old = 1e6
    
    for e in range(n_epoch) :
    
        total_training_loss = 0   
        model.train()
        for batch in train_loader :

            optimizer.zero_grad()   

            audio = batch[0]
            targets = batch[1]
            target_lengths = batch[2]        
            current_batch_size = audio.size()[0]
            output = model(audio)

            # this basically a tensor vector of the length the size of the current
            # batch size, each entry being the length of the predictions (determined in the model)
            input_lengths = torch.full(size=(current_batch_size,), fill_value=output.size()[-1], dtype=torch.long)

            # loss = ctc_loss(input, target, input_lengths, target_lengths)
            loss = criterion(output.transpose(1, 2).transpose(0, 1),targets,input_lengths,target_lengths)        
            total_training_loss += float(loss.cpu())

            loss.backward()
            optimizer.step()

        total_val_loss = 0  
        total_lev_dist = 0
        model.eval()      
        for batch in val_loader :

            audio = batch[0]
            targets = batch[1]
            target_lengths = batch[2]        
            current_batch_size = audio.size()[0]
            output = model(audio)        

            input_lengths = torch.full(size=(current_batch_size,), fill_value=output.size()[-1], dtype=torch.long)
            loss = criterion(output.transpose(1, 2).transpose(0, 1),targets,input_lengths,target_lengths)

            total_val_loss += float(loss.cpu())

            targets = targets.cpu().numpy().astype('int')     
            outmax = torch.argmax(output,dim=1).cpu().numpy()
            for i, vec in enumerate(outmax):

                original = postprocessor.target2kana(targets[i]) 
                predicted = postprocessor.target2kana(vec,refine = True)
                lev_dist = postprocessor.levenshtein(original,predicted)
                total_lev_dist += lev_dist/len(original)

        av_lev_dist = total_lev_dist/n_val

        if av_lev_dist < av_lev_dist_old :
            optimal_model = copy.deepcopy(model)
            av_lev_dist_old = copy.copy(av_lev_dist)

        print(e,total_training_loss/n_train,total_val_loss/n_val,av_lev_dist)
        
    return optimal_model

In [8]:
n_class = 79
model = Wav2Letter(n_class)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
criterion = torch.nn.CTCLoss(reduction='sum')
n_epoch = 200
trained_model1 = train(model,train_loader,val_loader,optimizer,criterion,n_epoch)

0 58.71476125628696 17.372999188099957 0.9954090150250415
1 16.29796502674223 15.057456346107445 1.0
2 13.667241644638146 12.152029199870878 1.0
3 10.23812191313848 8.844768619696564 0.8079431327344526
4 7.001251041645906 6.046804335758164 0.530683414155869


KeyboardInterrupt: 

In [46]:
def infer(model,val_loader) :
    model.eval()   
    total_lev_dist = 0
    n_val = len(val_loader.dataset)
    for data in val_loader :
        
        audio = data[0]
        targets = data[1].cpu().numpy().astype('int')     
        output = model(audio)
        outmax = torch.argmax(output,dim=1).cpu().numpy()
        
        for i, vec in enumerate(outmax):
            
            original = postprocessor.target2kana(targets[i]) 
            predicted = postprocessor.target2kana(vec,refine = True)
            lev_dist = postprocessor.levenshtein(original,predicted)
            total_lev_dist += lev_dist/len(original)
            
            print(original, predicted,lev_dist,round(lev_dist/len(original),2))
            
        av_lev = total_lev_dist/n_val
        
    print('average Levenshtein distance',av_lev)

In [62]:
infer(trained_model1,val_loader)

オネエサン オメイサ 3 0.6
ヤワラカ ヤワラカ 0 0.0
キレイ キレイ 0 0.0
テンサイ テンサイ 0 0.0
クミタテル クミカテル 1 0.2
タトエバ タトイバ 1 0.25
ジンブンカガク ジンブンクラク 2 0.29
トッサニ トッサニ 0 0.0
サツジン サツジン 0 0.0
ドーイ ドイ 1 0.33
オカス オカス 0 0.0
タハタ タハタ 0 0.0
ニシ ミシ 1 0.5
ガンジツ ガンジツ 0 0.0
カマウ カナン 2 0.67
ヒル ヒル 0 0.0
ハイセン ハイセン 0 0.0
ブンカ ブンカ 0 0.0
エンピツ エンキツ 1 0.25
ナメラカ ナメラカ 0 0.0
ホントーニ ホントーニ 0 0.0
ハンドル ハンドル 0 0.0
イダイ イダイ 0 0.0
レクリエーション レクリエーション 0 0.0
ブンルイ ブンルイ 0 0.0
ワイン ワイン 0 0.0
レストラン レストラン 0 0.0
オイシイ オイシイ 0 0.0
シンコー シンコー 0 0.0
ミズ ミズ 0 0.0
ドクトク ドクトク 0 0.0
センメン センネン 1 0.25
テツズキ テツズキ 0 0.0
ハッテン ハッテン 0 0.0
ニンズー ミンズン 2 0.5
セイジツ セイジツ 0 0.0
ウェーター ウェイター 1 0.2
イジ イジ 0 0.0
ギロン ギロン 0 0.0
キエル キエル 0 0.0
ハンイ ハンイ 0 0.0
ヨユー ヨユー 0 0.0
ミワタス ミワカス 1 0.25
トル トル 0 0.0
ハチ ハチ 0 0.0
スイジュン スイジュン 0 0.0
デザート エザート 1 0.25
ゴールデンウィーク ーレク 7 0.78
ベタベタ デタ 3 0.75
タクサン パクサン 1 0.25
ハダ ハダ 0 0.0
イソグ イソン 1 0.33
タンスー タンスー 0 0.0
ハツゲン ハツベン 1 0.25
センゾ センゾ 0 0.0
ミハラシ ミハラシ 0 0.0
センゼン センゼン 0 0.0
ミブリ ミブリ 0 0.0
ジシュー ジシュー 0 0.0
ボンチ ボンチ 0 0.0
ゲ ゲ 0 0.0
ゼンシン ゼンシン 0 0.0
ガッカリ タッカイ 2 0.5
テスト テスト 0 0.0
カタミチ 

In [ ]:
model_aug = Wav2Letter(n_class)
model_aug = model_aug.cuda()
optimizer = torch.optim.Adam(model_aug.parameters(),lr=1e-4)
criterion = torch.nn.CTCLoss(reduction='sum')

dataset_train_aug = AudioDataset(audio_list_train,target_list_train,n_audio_max,n_target_max,random_pad = True,change_speed=True)
dataset_val_aug = AudioDataset(audio_list_val,target_list_val,n_audio_max,n_target_max)

train_loader_aug = DataLoader(dataset_train_aug, batch_size=batch_size_train,shuffle=True)
val_loader_aug = DataLoader(dataset_val_aug, batch_size=batch_size_val)

n_epoch = 200
trained_model2 = train(model_aug,train_loader_aug,val_loader_aug,optimizer,criterion,n_epoch)
infer(trained_model2,val_loader_aug)

In [ ]:
infer(trained_model2,val_loader_aug)